<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
!pip install pandas==2.2.2 -qq
!pip install pyxirr -qq
!pip install ta -qq
!pip install yfinance==0.2.59 -qq
'''

'\n!pip install pandas==2.2.2 -qq\n!pip install pyxirr -qq\n!pip install ta -qq\n!pip install yfinance==0.2.59 -qq\n'

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  #stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-02-05 13:37:41
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  3.49 K
Current:  1.39 C
-------------------
Today PnL: -61.08 K (-0.44%)
Current PnL: -30.63 L (-19.53%)
CY Booked + Current PnL: -15.89 L (-10.13%)
-------------------
Total profit:  1.24 L
Total loss:  -31.87 L
-------------------
Total Booked + Current PnL: 11.09 L (8.54%)
Total Booked PnL: 41.72 L (32.14%)
Curr Year Booked PnL: 14.74 L (10.61%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.35 C
Est FTT PnL: 96.01 L (69.14%)
Deployed:  1.30 C
Current:  1.39 C
CAGR/XIRR %: 3.52%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COALINDIA,-1.23,5.63,12.87,19.23,20455.0,8474.0,158934.0,484.83,24.27,53.0,L-LC,12.33,182.0,0.41,1.14,28.11,XY25,ATH,MINING
50,MASFIN,1.27,-3.48,25.94,21.57,24533.0,-3405.0,94575.0,397.04,-16.89,51.0,H-SC,3.51,168.0,-0.14,0.68,38.01,XR,ATH,FINANCE
77,TTKPRESTIG,0.18,-27.77,38.58,0.09,28082.0,-27987.0,72790.0,770.00,69.05,34.0,M-SC,7.91,253.0,-1.00,0.52,0.18,OX40N,NTT,DURABLES
57,RECLTD,0.24,3.55,16.60,20.74,34922.0,7205.0,210375.0,446.00,35.83,60.0,M-MC,3.98,189.0,0.21,1.52,14.54,XY25,NTT,FINANCE
36,ICICIGI,1.15,-0.66,20.13,19.34,40017.0,-1317.0,198792.0,2252.93,-21.68,50.0,X-MC,3.46,71.0,-0.03,1.43,15.18,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,VALIANTORG,5.43,-46.38,224.79,74.15,197193.0,-75882.0,87723.0,838.00,-257.83,56.0,H-SC,26.44,145.0,-0.38,0.63,12.45,XR,NTT,CHEMICALS
14,BSOFT,3.66,-17.25,82.21,50.79,95688.0,-24258.0,116395.0,831.70,6.30,63.0,H-SC,6.56,171.0,-0.25,0.84,35.52,XR,ATH,IT
83,WHIRLPOOL,3.00,-34.34,177.08,81.92,133467.0,-39427.0,75371.0,2270.00,-61.52,45.0,M-SC,32.98,218.0,-0.30,0.54,5.87,XR,NTT,DURABLES
24,DMART,2.87,-5.43,37.04,29.60,87431.0,-13554.0,236046.0,5391.45,-19.34,63.0,X-LC,9.63,10.0,-0.16,1.70,15.57,X40N,ATH,FMCG
22,DEN,1.84,-44.33,155.19,42.07,72977.0,-37440.0,47024.0,75.00,68.39,49.0,M-SC,13.81,236.0,-0.51,0.34,8.01,AR,NTT,ENTERTAINMENT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COFFEEDAY,-3.82,-48.93,164.90,35.27,95616.0,-55565.0,57984.0,80.0,-61.77,38.0,L-SC,12.90,268.0,-0.58,0.42,41.92,XR,NTT,HOTELS
76,TRIDENT,-3.38,-24.28,69.73,28.51,48708.0,-22402.0,69852.0,48.0,-10.35,61.0,M-SC,9.93,226.0,-0.46,0.50,19.53,XR,NTT,TEXTILES
56,RAJOOENG,-3.03,-36.91,121.28,39.61,78432.0,-37830.0,64670.0,143.1,-57.37,48.0,H-SC,29.41,136.0,-0.48,0.47,8.34,AR,ATH,MISC
0,5PAISA,-2.95,-36.94,79.05,12.90,108914.0,-80717.0,137779.0,593.0,103.12,43.0,H-SC,5.88,173.0,-0.74,0.99,14.56,OX40N,NTT,FINANCE
64,SHALBY,-2.63,-34.24,90.23,25.10,107488.0,-62016.0,119127.0,327.0,775.60,42.0,M-SC,15.01,248.0,-0.58,0.86,4.34,XY24,NTT,HEALTHCARE


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,FINCABLES,0.21,-5.44,119.88,107.91,167358.0,-8039.0,139605.0,1641.55,-22.98,51.0,M-SC,11.50,220.0,-0.05,1.01,5.03,OX40N,ATH,CABLES
66,SIS,1.49,-19.49,50.62,21.26,45249.0,-21642.0,89390.0,528.00,2115.03,66.0,H-SC,4.66,163.0,-0.48,0.64,20.78,OX40N,NTT,MISC
72,TATAELXSI,-1.60,-21.86,68.96,32.03,71041.0,-28817.0,103018.0,9161.00,-15.37,50.0,H-SC,6.38,157.0,-0.41,0.74,15.39,OX40N,NTT,IT
48,KANSAINER,-1.56,-27.04,55.54,13.47,109269.0,-72927.0,196740.0,340.00,-70.06,37.0,H-SC,5.57,159.0,-0.67,1.42,0.04,XY24,NTT,PAINTS
77,TTKPRESTIG,0.18,-27.77,38.58,0.09,28082.0,-27987.0,72790.0,770.00,69.05,34.0,M-SC,7.91,253.0,-1.00,0.52,0.18,OX40N,NTT,DURABLES


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RECLTD,0.24,3.55,16.60,20.74,34922.0,7205.0,210375.0,446.00,35.83,60.0,M-MC,3.98,189.0,0.21,1.52,14.54,XY25,NTT,FINANCE
18,COALINDIA,-1.23,5.63,12.87,19.23,20455.0,8474.0,158934.0,484.83,24.27,53.0,L-LC,12.33,182.0,0.41,1.14,28.11,XY25,ATH,MINING
1,ABB,1.05,6.70,36.43,45.57,101692.0,17525.0,279144.0,7934.00,-31.40,73.0,H-MC,5.15,121.0,0.17,2.01,23.96,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,1.05,6.70,36.43,45.57,101692.0,17525.0,279144.0,7934.00,-31.40,73.0,H-MC,5.15,121.0,0.17,2.01,23.96,AR,NTT,ELECTRICAL
39,INDIAMART,-0.05,-2.29,111.56,106.72,134449.0,-2819.0,120517.0,4810.62,-51.58,57.0,H-SC,8.49,138.0,-0.02,0.87,20.56,AR,ATH,MISC
26,FINCABLES,0.21,-5.44,119.88,107.91,167358.0,-8039.0,139605.0,1641.55,-22.98,51.0,M-SC,11.50,220.0,-0.05,1.01,5.03,OX40N,ATH,CABLES
84,WIPRO,-0.22,-4.50,80.39,72.27,130085.0,-7631.0,161817.0,420.00,-17.88,38.0,M-LC,2.75,101.0,-0.06,1.17,4.01,XR,NTT,IT
85,ZYDUSLIFE,0.31,-4.24,41.63,35.63,83547.0,-8884.0,200690.0,1286.17,-17.63,55.0,H-MC,6.05,119.0,-0.11,1.45,11.73,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BATAINDIA,-0.81,-44.95,145.61,35.20,103138.0,-57838.0,70832.0,2096.00,-7.15,25.0,X-SC,20.90,93.0,-0.56,0.51,0.66,X40,NTT,FOOTWEAR
44,ITC,-1.18,-23.98,45.74,10.78,83557.0,-57640.0,182678.0,452.00,-54.64,27.0,X-LC,18.82,1.0,-0.69,1.32,0.62,X40,NTT,FMCG
81,VBL,-1.51,-11.48,53.27,35.67,148928.0,-36270.0,279572.0,671.64,-22.02,27.0,X-LC,0.29,5.0,-0.24,2.01,0.78,X40N,ATH,FMCG
6,ASIANPAINT,-1.01,-15.14,42.52,20.95,90846.0,-38113.0,213655.0,3460.25,-15.95,29.0,X-LC,9.53,36.0,-0.42,1.54,15.10,X40,ATH,PAINTS
13,BERGEPAINT,-1.28,-15.57,43.48,21.14,83460.0,-35389.0,191950.0,680.00,-21.59,32.0,X-MC,8.45,75.0,-0.42,1.38,2.39,XY24,NTT,PAINTS


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,VBL,-1.51,-11.48,53.27,35.67,148928.0,-36270.0,279572.0,671.64,-22.02,27.0,X-LC,0.29,5.0,-0.24,2.01,0.78,X40N,ATH,FMCG
73,TCS,-0.16,-17.67,44.00,18.55,137015.0,-66840.0,311397.0,4311.59,-29.13,37.0,X-LC,0.48,3.0,-0.49,2.24,5.94,X40,ATH,IT
5,ANGELONE,-1.79,8.05,14.95,24.20,49309.0,24575.0,329825.0,3033.00,53.88,56.0,X-SC,1.44,99.0,0.50,2.38,38.14,X40N,NTT,FINANCE
34,HINDUNILVR,-0.20,-7.95,23.48,13.67,55561.0,-20433.0,236630.0,2922.00,-14.78,49.0,X-LC,2.35,9.0,-0.37,1.70,11.71,XY25,NTT,FMCG
21,DABUR,0.65,-1.37,45.70,43.70,111809.0,-3410.0,244658.0,735.00,-9.42,45.0,X-MC,2.39,73.0,-0.03,1.76,13.47,XY24,BTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,ITC,-1.18,-23.98,45.74,10.78,83557.0,-57640.0,182678.0,452.00,-54.64,27.0,X-LC,18.82,1.0,-0.69,1.32,0.62,X40,NTT,FMCG
12,BATAINDIA,-0.81,-44.95,145.61,35.20,103138.0,-57838.0,70832.0,2096.00,-7.15,25.0,X-SC,20.90,93.0,-0.56,0.51,0.66,X40,NTT,FOOTWEAR
81,VBL,-1.51,-11.48,53.27,35.67,148928.0,-36270.0,279572.0,671.64,-22.02,27.0,X-LC,0.29,5.0,-0.24,2.01,0.78,X40N,ATH,FMCG
2,ABBOTINDIA,0.00,-9.09,29.87,18.07,48569.0,-16255.0,162600.0,35195.00,-22.07,41.0,X-MC,9.32,64.0,-0.33,1.17,1.75,X40,ATH,PHARMA
53,PGHH,0.14,-11.35,48.90,32.01,92566.0,-24224.0,189296.0,17616.87,-37.61,41.0,X-MC,8.57,60.0,-0.26,1.36,2.32,X40,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,0.99,-26.17,93.47,42.84,44860.0,-17012.0,47994.0,424.00,-51.52,62.0,X-SC,21.15,83.0,-0.38,0.35,12.81,XY24,NTT,MISC
58,RELAXO,-1.16,-51.87,216.43,52.31,151220.0,-75290.0,69870.0,1176.00,-48.79,42.0,X-SC,12.35,92.0,-0.50,0.50,3.90,X40N,NTT,FOOTWEAR
12,BATAINDIA,-0.81,-44.95,145.61,35.20,103138.0,-57838.0,70832.0,2096.00,-7.15,25.0,X-SC,20.90,93.0,-0.56,0.51,0.66,X40,NTT,FOOTWEAR
51,MEDANTA,-2.35,-8.45,35.53,24.08,41292.0,-10732.0,116218.0,1486.00,-15.92,46.0,X-SC,11.16,91.0,-0.26,0.84,6.45,XY24,NTT,HEALTHCARE
35,HONAUT,-1.15,-19.84,78.79,43.32,102868.0,-32312.0,130560.0,58357.33,-31.14,46.0,X-SC,9.17,90.0,-0.31,0.94,6.01,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,ITC,-1.18,-23.98,45.74,10.78,83557.0,-57640.0,182678.0,452.00,-54.64,27.0,X-LC,18.82,1.0,-0.69,1.32,0.62,X40,NTT,FMCG
75,TMPV,-0.52,-23.45,60.64,22.98,104865.0,-52968.0,172930.0,600.00,-20.63,64.0,X-LC,4.33,2.0,-0.51,1.25,10.55,XY24,NTT,AUTO
73,TCS,-0.16,-17.67,44.00,18.55,137015.0,-66840.0,311397.0,4311.59,-29.13,37.0,X-LC,0.48,3.0,-0.49,2.24,5.94,X40,ATH,IT
81,VBL,-1.51,-11.48,53.27,35.67,148928.0,-36270.0,279572.0,671.64,-22.02,27.0,X-LC,0.29,5.0,-0.24,2.01,0.78,X40N,ATH,FMCG
42,INFY,-1.01,3.50,29.71,34.24,96208.0,10936.0,323824.0,1972.00,-19.24,33.0,X-LC,6.50,6.0,0.11,2.33,12.07,X40,NTT,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INDUSINDBK,-1.77,-33.19,98.91,32.89,51020.0,-25624.0,51582.0,1800.00,-736.93,52.0,L-MC,11.17,259.0,-0.50,0.37,42.05,XR,NTT,BANKS
67,SONACOMS,-1.14,-8.87,52.62,39.09,48510.0,-8969.0,92190.0,804.02,-28.76,70.0,M-SC,1.84,227.0,-0.18,0.66,30.42,AR,ATH,AUTO
14,BSOFT,3.66,-17.25,82.21,50.79,95688.0,-24258.0,116395.0,831.70,6.30,63.0,H-SC,6.56,171.0,-0.25,0.84,35.52,XR,ATH,IT
18,COALINDIA,-1.23,5.63,12.87,19.23,20455.0,8474.0,158934.0,484.83,24.27,53.0,L-LC,12.33,182.0,0.41,1.14,28.11,XY25,ATH,MINING
65,SIEMENS,0.67,-10.99,41.02,25.51,67943.0,-20460.0,165635.0,4671.50,1.56,69.0,H-LC,3.58,50.0,-0.30,1.19,22.06,AR,ATH,ELECTRICAL


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,1.05,6.70,36.43,45.57,101692.0,17525.0,279144.0,7934.00,-31.40,73.0,H-MC,5.15,121.0,0.17,2.01,23.96,AR,NTT,ELECTRICAL
5,ANGELONE,-1.79,8.05,14.95,24.20,49309.0,24575.0,329825.0,3033.00,53.88,56.0,X-SC,1.44,99.0,0.50,2.38,38.14,X40N,NTT,FINANCE
67,SONACOMS,-1.14,-8.87,52.62,39.09,48510.0,-8969.0,92190.0,804.02,-28.76,70.0,M-SC,1.84,227.0,-0.18,0.66,30.42,AR,ATH,AUTO
62,SATIN,0.72,-11.56,71.68,51.83,175149.0,-31936.0,244348.0,274.00,-21.16,65.0,H-SC,3.26,148.0,-0.18,1.76,19.03,XY24,NTT,FINANCE
24,DMART,2.87,-5.43,37.04,29.60,87431.0,-13554.0,236046.0,5391.45,-19.34,63.0,X-LC,9.63,10.0,-0.16,1.70,15.57,X40N,ATH,FMCG


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.33
1,25,44.68
2,50,76.72


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.33
MC    30.90
LC    23.77
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.58
X40      23.40
X40N     15.07
AR        9.09
XR        8.79
XY25      8.78
OX40N     7.39
SR        0.90
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.09
H-SC    22.96
X-LC    20.27
M-SC    11.28
X-SC    10.33
H-MC     4.92
M-MC     1.52
H-LC     1.19
M-LC     1.17
L-LC     1.14
L-SC     0.76
L-MC     0.37
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.89,-14.49,52.95
FINANCE,13.53,-11.21,54.98
IT,12.42,-24.02,85.98
MISC,7.21,-29.86,82.97
ELECTRICAL,6.18,-9.32,49.21
PAINTS,5.24,-26.84,45.69
INSURANCE,4.80,-1.75,36.57
PHARMA,3.99,-6.12,39.53
AUTO,2.97,-27.03,70.96


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3280712.0,21
AR,1310110.0,10
XR,1271792.0,12
X40,1250068.0,15
X40N,1028829.0,10
OX40N,763407.0,10
XY25,400839.0,6
SR,295633.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3607933.0,24
X-MC,1682384.0,16
M-SC,1488507.0,15
X-LC,1077086.0,12
X-SC,859632.0,9
H-MC,396263.0,3
L-SC,185160.0,2
M-LC,130085.0,1
H-LC,67943.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1237191.0      6
           AR         935441.0      5
M-SC       XY24       837754.0      6
H-SC       XR         812896.0      7
X-MC       X40        562777.0      7
X-LC       X40        513843.0      6
X-MC       XY24       480027.0      3
           X40N       415474.0      4
X-SC       X40N       376996.0      4
M-SC       OX40N      347091.0      5
H-SC       OX40N      326772.0      4
X-SC       XY24       309188.0      3
H-SC       SR         295633.0      2
X-LC       X40N       236359.0      2
X-MC       XY25       224106.0      2
H-MC       XY24       211024.0      1
X-LC       XY24       205528.0      2
H-MC       AR         185239.0      2
M-SC       XR         182175.0      2
X-SC       X40        173448.0      2
M-LC       XR         130085.0      1
M-SC       AR         121487.0      2
X-LC       XY25       121356.0      2
L-SC       XR          95616.0      1
           OX40N       89544.0      1
H-LC       AR          67943.0      1
L-MC       XR          51020.0      1
M-MC       XY25        34922.0      1
L-LC       XY25        20455.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 41.0 seconds
